Let's try some basic tests of `vbgraph.jl`:

In [1]:
include("vbgraph.jl")

Let's try to make a Gaussian node with several different sizes and parameterizations:

In [2]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

3

In [3]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [4]:
μ[1]

3-element Array{Float64,1}:
 0.150363
 0.940933
 0.360343

In [5]:
Σ[1]

3x3 Array{Float64,2}:
 0.9477    0.707345  1.12777
 0.707345  0.942082  1.13008
 1.12777   1.13008   1.60086

In [6]:
det(Σ[2])

0.0007367037796127104

In [7]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...)

5x5 Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}:
 FullNormal(
dim: 3
μ: [0.15036283770005543,0.9409328520444713,0.3603433478233937]
Σ: 3x3 Array{Float64,2}:
 0.9477    0.707345  1.12777
 0.707345  0.942082  1.13008
 1.12777   1.13008   1.60086
)
     …  FullNormal(
dim: 3
μ: [0.6216528932467331,0.25992317009623234,0.9131081367176188]
Σ: 3x3 Array{Float64,2}:
 1.62215   1.04346   0.538243
 1.04346   1.70547   0.368588
 0.538243  0.368588  0.261057
)

 FullNormal(
dim: 3
μ: [0.701350127045336,0.5144044472558258,0.24229430092094795]
Σ: 3x3 Array{Float64,2}:
 0.457368  0.650166  0.815844
 0.650166  0.956508  1.17558 
 0.815844  1.17558   1.51296 
)
      FullNormal(
dim: 3
μ: [0.38219511544506535,0.9453906205138853,0.21124208015455959]
Σ: 3x3 Array{Float64,2}:
 0.23745   0.407955  0.57381
 0.407955  0.746279  1.01247
 0.57381   1.01247   1.43159
)
  
 FullNormal(
dim: 3
μ: [0.4333416496895155,0.656419059463901,0.3118475896207493]
Σ: 3x3 Array{Float64,2}:
 1.22776   0.870

In [8]:
rr = RandomNode(aa, Factor[])

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{RandomNode{D<:Distributions.Distribution{F<:Distributions.VariateForm,S<:Distributions.ValueSupport},N,A<:AbstractArray{T,N}}}, ::Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}, ::Array{Factor,1})
This may have arisen from a call to the constructor RandomNode{D<:Distributions.Distribution{F<:Distributions.VariateForm,S<:Distributions.ValueSupport},N,A<:AbstractArray{T,N}}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[8], in expression starting on line 1

In [9]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [10]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [11]:
ss = RandomNode(MultivariateNormal, pars...)

RandomNode{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}(5x5 Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}:
 FullNormal(
dim: 3
μ: [0.17511908291078537,0.9250255394812403,0.4328455990818858]
Σ: 3x3 Array{Float64,2}:
 0.865937  1.03344   0.580606
 1.03344   1.26273   0.786931
 0.580606  0.786931  0.839992
)
   …  FullNormal(
dim: 3
μ: [0.4572421699676201,0.9910682583324493,0.3588825998739198]
Σ: 3x3 Array{Float64,2}:
 1.13069   0.154043   0.925126
 0.154043  0.0303103  0.117927
 0.925126  0.117927   1.63713 
)

 FullNormal(
dim: 3
μ: [0.8952293595054748,0.20523356770212087,0.6902354907277803]
Σ: 3x3 Array{Float64,2}:
 0.456633  0.627108  0.627088
 0.627108  0.984303  0.69987 
 0.627088  0.69987   1.21318 
)
      FullNormal(
dim: 3
μ: [0.8791630447235963,0.9446592392229962,0.6897791575611507]
Σ: 3x3 Array{Float64,2}:
 0.795506  0.568989  1.07804
 0.568989  0.728935  1.00827
 1.07804   1.00827   1.63713
)
      
 FullNormal(
dim: 3
μ: [0.11390987828380061,0.06